[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/04-chat-memory.ipynb)

### LangChain Essentials

# Conversational Memory for OpenAI - LangChain

Conversational memory allows our chatbots and agents to remember previous interactions within a conversation. Without conversational memory, our chatbots would only ever be able to respond to the last message they received, essentially forgetting all previous messages with each new message.

Naturally, conversations require our chatbots to be able to respond over multiple interactions and refer to previous messages to understand the context of the conversation.

In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16 \
  langsmith==0.3.4

---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [1]:
from IPython.display import display, Markdown
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-chat-memory-openai"

---

## LangChain's Memory Types

LangChain versions `0.0.x` consisted of various conversational memory types. Most of these are due for deprecation but still hold value in understanding the different approaches that we can take to building conversational memory.

Throughout the notebook we will be referring to these _older_ memory types and then rewriting them using the recommended `RunnableWithMessageHistory` class. We will learn about:

* `ConversationBufferMemory`: the simplest and most intuitive form of conversational memory, keeping track of a conversation without any additional bells and whistles.
* `ConversationBufferWindowMemory`: similar to `ConversationBufferMemory`, but only keeps track of the last `k` messages.
* `ConversationSummaryMemory`: rather than keeping track of the entire conversation, this memory type keeps track of a summary of the conversation.
* `ConversationSummaryBufferMemory`: merges the `ConversationSummaryMemory` and `ConversationTokenBufferMemory` types.

We'll work through each of these memory types in turn, and rewrite each one using the `RunnableWithMessageHistory` class.

## Initialize our LLM

Before jumping into our memory types, let's initialize our LLM. We will use OpenAI's `gpt-4o-mini` model, if you need an API key you can get one from [OpenAI's website](https://platform.openai.com/settings/organization/api-keys).

In [2]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or \
    getpass("Enter your OpenAI API key: ")

# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

## 1. `ConversationBufferMemory`

`ConversationBufferMemory` is the simplest form of conversational memory, it is _literally_ just a place that we store messages, and then use to feed messages into our LLM.

Let's start with LangChain's original `ConversationBufferMemory` object, we are setting `return_messages=True` to return the messages as a list of `ChatMessage` objects — unless using a non-chat model we would always set this to `True` as without it the messages are passed as a direct string which can lead to unexpected behavior from chat LLMs.

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

C:\Users\rishi\AppData\Local\Temp\ipykernel_16280\1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


There are several ways that we can add messages to our memory, using the `save_context` method we can add a user query (via the `input` key) and the AI's response (via the `output` key). So, to create the following conversation:

```
User: Hi, my name is James
AI: Hey James, what's up? I'm an AI model called Zeta.
User: I'm researching the different types of conversational memory.
AI: That's interesting, what are some examples?
User: I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.
AI: That's interesting, what's the difference?
User: Buffer memory just stores the entire conversation, right?
AI: That makes sense, what about ConversationBufferWindowMemory?
User: Buffer window memory stores the last k messages, dropping the rest.
AI: Very cool!
```

We do:

In [4]:
memory.save_context(
    {"input": "Hi, my name is James"},  # user message
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}  # AI response
)
memory.save_context(
    {"input": "I'm researching the different types of conversational memory."},  # user message
    {"output": "That's interesting, what are some examples?"}  # AI response
)
memory.save_context(
    {"input": "I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory."},  # user message
    {"output": "That's interesting, what's the difference?"}  # AI response
)
memory.save_context(
    {"input": "Buffer memory just stores the entire conversation, right?"},  # user message
    {"output": "That makes sense, what about ConversationBufferWindowMemory?"}  # AI response
)
memory.save_context(
    {"input": "Buffer window memory stores the last k messages, dropping the rest."},  # user message
    {"output": "Very cool!"}  # AI response
)

Before using the memory, we need to load in any variables for that memory type — in this case, there are none, so we just pass an empty dictionary:

In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

With that, we've created our buffer memory. Before feeding it into our LLM let's quickly view the alternative method for adding messages to our memory. With this other method, we pass individual user and AI messages via the `add_user_message` and `add_ai_message` methods. To reproduce what we did above, we do:

In [6]:
memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message("Hi, my name is James")
memory.chat_memory.add_ai_message("Hey James, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

The outcome is exactly the same in either case. To pass this onto our LLM, we need to create a `ConversationChain` object — which is already deprecated in favor of the `RunnableWithMessageHistory` class, which we will cover in a moment.

In [7]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

C:\Users\rishi\AppData\Local\Temp\ipykernel_16280\839683240.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(


In [8]:
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kw

{'input': 'what is my name again?',
 'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}

### `ConversationBufferMemory` with `RunnableWithMessageHistory`

As mentioned, the `ConversationBufferMemory` type is due for deprecation. Instead, we can use the `RunnableWithMessageHistory` class to implement the same functionality.

When implementing `RunnableWithMessageHistory` we will use **L**ang**C**hain **E**xpression **L**anguage (LCEL) and for this we need to define our prompt template and LLM components. Our `llm` has already been defined, so now we just define a `ChatPromptTemplate` object.

In [10]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

We can link our `prompt_template` and our `llm` together to create a pipeline via LCEL.

In [11]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template 
    | llm
    | {"First-Response": lambda x: x.content}
)

Our `RunnableWithMessageHistory` requires our `pipeline` to be wrapped in a `RunnableWithMessageHistory` object. This object requires a few input parameters. One of those is `get_session_history`, which requires a function that returns a `ChatMessageHistory` object based on a session ID. We define this function ourselves:

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

We also need to tell our runnable which variable name to use for the chat history (ie `history`) and which to use for the user's query (ie `query`).

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

Now we invoke our runnable:

In [14]:
out = pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123"}
)
print(out["First-Response"])

Hi again, James! How can I help you today?


Our chat history will now be memorized and retrieved whenever we invoke our runnable with the same session ID.

In [15]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

{'First-Response': 'Your name is James. How can I assist you further?'}

We have now recreated the `ConversationBufferMemory` type using the `RunnableWithMessageHistory` class. Let's continue onto other memory types and see how these can be implemented.

## 2. `ConversationBufferWindowMemory`

The `ConversationBufferWindowMemory` type is similar to `ConversationBufferMemory`, but only keeps track of the last `k` messages. There are a few reasons why we would want to keep only the last `k` messages:

* More messages mean more tokens are sent with each request, more tokens increases latency _and_ cost.

* LLMs tend to perform worse when given more tokens, making them more likely to deviate from instructions, hallucinate, or _"forget"_ information provided to them. Conciseness is key to high performing LLMs.

* If we keep _all_ messages we will eventually hit the LLM's context window limit, by adding a window size `k` we can ensure we never hit this limit.

The buffer window solves many problems that we encounter with the standard buffer memory, while still being a very simple and intuitive form of conversational memory.

In [18]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

C:\Users\rishi\AppData\Local\Temp\ipykernel_22116\3216785012.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=4, return_messages=True)


We populate this memory using the same methods as before:

In [19]:
memory.chat_memory.add_user_message("Hi, my name is James")
memory.chat_memory.add_ai_message("Hey James, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

As before, we use the `ConversationChain` object (again, this is deprecated and we will rewrite it with `RunnableWithMessageHistory` in a moment).

In [20]:
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

Now let's see if our LLM remembers our name:

In [21]:
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='That makes sense, what about 

{'input': 'what is my name again?',
 'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kw

The reason our LLM can no longer remember our name is because we have set the `k` parameter to `4`, meaning that only the last messages are stored in memory, as we can see above this does not include the first message where we introduced ourselves.

Based on the agent forgetting our name, we might wonder _why_ we would ever use this memory type compared to the standard buffer memory. Well, as with most things in AI, it is always a trade-off. Here we are able to support much longer conversations, use less tokens, and improve latency — but these come at the cost of forgetting non-recent messages.

### `ConversationBufferWindowMemory` with `RunnableWithMessageHistory`

To implement this memory type using the `RunnableWithMessageHistory` class, we can use the same approach as before. We define our `prompt_template` and `llm` as before, and then wrap our pipeline in a `RunnableWithMessageHistory` object.

For the window feature, we need to define a custom version of the `InMemoryChatMessageHistory` class that removes any messages beyond the last `k` messages.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        if len(self.messages) > self.k: # Add logging to help with debugging
            print(f"Truncating history from {len(self.messages)} to {self.k} messages")
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [23]:
chat_map = {}
def get_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    print(f"get_chat_history called with session_id={session_id} and k={k}")
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    # remove anything beyond the last
    return chat_map[session_id]

In [14]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

Now we invoke our runnable, this time passing a `k` parameter via the `config` parameter.

In [30]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4
Initializing BufferWindowMessageHistory with k=4


{'First-Response': 'Hi James! How can I assist you today?'}

We can also modify the messages that are stored in memory by modifying the records inside the `chat_map` dictionary directly.

In [31]:
chat_map["id_k4"].clear()  # clear the history

# manually insert history
chat_map["id_k4"].add_user_message("Hi, my name is James")
chat_map["id_k4"].add_ai_message("I'm an AI model called Zeta.")
chat_map["id_k4"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k4"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k4"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k4"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k4"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k4"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k4"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k4"].add_ai_message("Very cool!")

chat_map["id_k4"].messages

[HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]

Now let's see at which `k` value our LLM remembers our name — from the above we can already see that with `k=4` our name is not mentioned, so when running with `k=4` we should expect the LLM to forget our name:

In [32]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4


{'First-Response': "I'm sorry, but I don't have access to personal information about users unless you've shared it in this conversation. If you'd like me to remember your name, please tell me!"}

Now let's initialize a new session with `k=14`.

In [33]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14
Initializing BufferWindowMessageHistory with k=14


{'First-Response': 'Hi James! How can I assist you today?'}

We'll manually insert the remaining messages as before:

In [34]:
chat_map["id_k14"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k14"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k14"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k14"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k14"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k14"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k14"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k14"].add_ai_message("Very cool!")

chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory

Now let's see if the LLM remembers our name:

In [35]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"session_id": "id_k14", "k": 14}
)

get_chat_history called with session_id=id_k14 and k=14


{'First-Response': 'Your name is James.'}

In [36]:
chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory

That's it! We've rewritten our buffer window memory using the recommended `RunnableWithMessageHistory` class.

## 3. `ConversationSummaryMemory`

Next up we have `ConversationSummaryMemory`, this memory type keeps track of a summary of the conversation rather than the entire conversation. This is useful for long conversations where we don't need to keep track of the entire conversation, but we do want to keep some thread of the full conversation.

As before, we'll start with the original memory class before reimplementing it with the `RunnableWithMessageHistory` class.

In [37]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)

C:\Users\rishi\AppData\Local\Temp\ipykernel_22116\988334424.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


Unlike with the previous memory types, we need to provide an `llm` to initialize `ConversationSummaryMemory`. The reason for this is that we need an LLM to generate the conversation summaries.

Beyond this small tweak, using `ConversationSummaryMemory` is the same as with our previous memory types when using the deprecated `ConversationChain` object.

In [38]:
chain = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

Let's test:

In [39]:
chain.invoke({"input": "hello there my name is James"})
chain.invoke({"input": "I am researching the different types of conversational memory."})
chain.invoke({"input": "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory."})
chain.invoke({"input": "Buffer memory just stores the entire conversation"})
chain.invoke({"input": "Buffer window memory stores the last k messages, dropping the rest."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there my name is James
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as James. The AI greets James warmly and offers to chat and help with anything he needs, inquiring about how his day is going.
Human: I am researching the different types of conversational memory.
AI:

> Finished chain.


> Entering new ConversationChain chain...
P

{'input': 'Buffer window memory stores the last k messages, dropping the rest.',
 'history': 'The human introduces himself as James, and the AI greets him warmly, offering to chat and help with anything he needs while inquiring about his day. James mentions he is researching different types of conversational memory, to which the AI responds by explaining various types: short-term memory for context within a single conversation, long-term memory for storing information across multiple interactions, contextual memory for understanding conversation flow, dynamic memory for adapting to new information, and episodic memory for recalling specific events. The AI then asks if James is looking into a specific type of conversational memory or exploring them all. James shares that he has been looking at ConversationBufferMemory and ConversationBufferWindowMemory. The AI describes ConversationBufferMemory as a type that keeps track of recent exchanges to maintain context and continuity, while Conv

We can see how the conversation summary varies with each new message. Let's see if the LLM is able to recall our name:

In [40]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as James, and the AI greets him warmly, offering to chat and help with anything he needs while inquiring about his day. James mentions he is researching different types of conversational memory, to which the AI responds by explaining various types: short-term memory for context within a single conversation, long-term memory for storing information across multiple interactions, contextual memory for understanding conversation flow, dynamic memory for adapting to new information, and episodic memory for recalling specific events. The AI then asks if James is looking into a specific type of conversational memory or exploring them all. James sh

{'input': 'What is my name again?',
 'history': 'The human introduces himself as James, and the AI greets him warmly, offering to chat and help with anything he needs while inquiring about his day. James mentions he is researching different types of conversational memory, to which the AI responds by explaining various types: short-term memory for context within a single conversation, long-term memory for storing information across multiple interactions, contextual memory for understanding conversation flow, dynamic memory for adapting to new information, and episodic memory for recalling specific events. The AI then asks if James is looking into a specific type of conversational memory or exploring them all. James shares that he has been looking at ConversationBufferMemory and ConversationBufferWindowMemory. The AI describes ConversationBufferMemory as a type that keeps track of recent exchanges to maintain context and continuity, while ConversationBufferWindowMemory focuses on a slidi

As this information was stored in the summary the LLM successfully recalled our name. This may not always be the case, by summarizing the conversation we inevitably compress the full amount of information and so we may lose key details occasionally. Nonetheless, this is a great memory type for long conversations while retaining some key information.

### `ConversationSummaryMemory` with `RunnableWithMessageHistory`

Let's implement this memory type using the `RunnableWithMessageHistory` class. As with the window buffer memory, we need to define a custom implementation of the `InMemoryChatMessageHistory` class. We'll call this one `ConversationSummaryMessageHistory`.

In [50]:
from langchain_core.messages import SystemMessage


class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)

    def __init__(self, llm: ChatOpenAI):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Create a new summary of the conversation
        given the existing summary and the new messages.
        """
        self.messages.extend(messages)
        # check if there is an existing summary and extract it
        # if not, initialize an empty summary
        existing_summary = ""
        if self.messages and isinstance(self.messages[0], SystemMessage):
            existing_summary = self.messages[0].content
        # create a prompt to summarize the conversation
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        # Format the messages and invoke the LLM to generate a new summary
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=existing_summary,
                messages=[x.content for x in messages]
            )
        )
        # Replace the existing history with a new single system summary message
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [51]:
chat_map = {}
def get_chat_history(session_id: str, llm: ChatOpenAI) -> ConversationSummaryMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    # return the chat history
    return chat_map[session_id]

In [52]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
    ]
)

Now we invoke our runnable, this time passing a `llm` parameter via the `config` parameter.

In [53]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123_with_summary", "llm": llm}
)

{'First-Response': 'Hi James! How can I assist you today?'}

Let's see what summary was generated:

In [54]:
chat_map["id_123_with_summary"].messages

[SystemMessage(content='James introduced himself, and the assistant responded warmly, asking how it could assist him today.', additional_kwargs={}, response_metadata={})]

Let's continue the conversation and see if the summary is updated:

In [ ]:
pipeline_with_history.invoke(
    {"query": "I'm researching the different types of conversational memory."},
    config={"session_id": "id_123_with_summary", "llm": llm}
)

display(Markdown(chat_map["id_123_with_summary"].messages[0].content))

James introduced himself, and the assistant responded warmly, inquiring how it could assist him. James mentioned that he is researching different types of conversational memory. The assistant expressed interest in the topic and outlined several types of conversational memory, including:

1. **Short-term Memory**: Retaining information briefly during conversations.
2. **Long-term Memory**: Storing information over extended periods to recall past interactions.
3. **Contextual Memory**: Remembering the context of conversations, including setting and emotional tone.
4. **Social Memory**: The collective memory of a group that influences conversation dynamics.
5. **Conversational Memory in AI**: The ability of AI systems to remember past interactions for personalized responses.
6. **Episodic Memory**: Recollection of specific events or experiences from past discussions.
7. **Semantic Memory**: Storage of general knowledge and facts relevant to conversations.

The assistant invited James to specify any particular aspect of conversational memory he would like to explore further.

So far so good! Let's continue with a few more messages before returning to the name question.

In [61]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    pipeline_with_history.invoke(
        {"query": msg},
        config={"session_id": "id_123_with_summary", "llm": llm}
    )

Let's see the latest summary:

In [62]:
display(Markdown(chat_map["id_123_with_summary"].messages[0].content))

In the conversation, James introduced himself and discussed his research focus on various types of conversational memory. The assistant expressed interest and elaborated on several types, including short-term, long-term, contextual, social, conversational memory in AI, episodic, and semantic memory. James specifically highlighted his interest in **ConversationBufferMemory** and **ConversationBufferWindowMemory**.

The assistant provided an overview of both concepts:

1. **ConversationBufferMemory**: This type retains the entire conversation history in a buffer, allowing the AI to access all previous exchanges during the current session. It is particularly useful for maintaining context in applications like customer support or therapy chatbots.

2. **ConversationBufferWindowMemory**: This operates on a sliding window principle, storing only a limited number of the most recent exchanges (the last \( k \) messages), which makes it more efficient in memory usage for scenarios where only the latest context is needed.

James confirmed that buffer memory stores the entire conversation, and the assistant agreed, discussing the advantages and disadvantages of **ConversationBufferMemory**:

### Advantages:
- **Contextual Awareness**: The AI can reference any part of the conversation, aiding in understanding user intent and providing relevant responses.
- **Rich Interaction**: It allows for more nuanced interactions as the AI can build on previous exchanges.

### Disadvantages:
- **Memory Usage**: Storing the entire conversation can lead to high memory consumption, especially in lengthy interactions.
- **Performance**: As the conversation length increases, retrieving and processing the entire history may slow down response times.

The assistant then detailed the advantages and disadvantages of **ConversationBufferWindowMemory**:

### Advantages:
- **Efficiency**: By limiting the stored messages, it reduces memory consumption, making it suitable for applications with resource constraints.
- **Speed**: With fewer messages to process, the AI can respond more quickly, which is beneficial in real-time interactions.

### Disadvantages:
- **Loss of Context**: Important information from earlier in the conversation may be lost if it falls outside the window, potentially leading to misunderstandings or less relevant responses.
- **Limited Historical Insight**: The AI may not be able to reference earlier parts of the conversation that could be crucial for understanding the user's intent or providing comprehensive answers.

The assistant encouraged James to share specific scenarios where either type of memory might be particularly beneficial or challenging.

The information about our name has been maintained, so let's see if this is enough for our LLM to correctly recall our name.

In [63]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123_with_summary", "llm": llm}
)

{'First-Response': 'Your name is James.'}

Perfect! We've successfully implemented the `ConversationSummaryMemory` type using the `RunnableWithMessageHistory` class.

## 4. `ConversationSummaryBufferMemory`

Our final memory type acts as a combination of `ConversationSummaryMemory` and `ConversationBufferMemory`. It keeps the buffer for the conversation up until the previous `n` tokens, anything beyond that limit is summarized then dropped from the buffer. Producing something like:


```
# ~~ a summary of previous interactions
The user named James introduced himself and the AI responded, introducing itself as an AI model called Zeta.
James then said he was researching the different types of conversational memory and Zeta asked for some
examples.
# ~~ the most recent messages
Human: I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.
AI: That's interesting, what's the difference?
Human: Buffer memory just stores the entire conversation
AI: That makes sense, what about ConversationBufferWindowMemory?
Human: Buffer window memory stores the last k messages, dropping the rest.
AI: Very cool!
```


In [64]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=300,
    return_messages=True
)

C:\Users\rishi\AppData\Local\Temp\ipykernel_22116\569741439.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


As before, we set up the deprecated memory type using the `ConversationChain` object.

In [65]:
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

First invoke with a single message:

In [66]:
chain.invoke({"input": "Hi, my name is James"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, my name is James
AI:

> Finished chain.


{'input': 'Hi, my name is James',
 'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello, James! It's great to meet you! How's your day going so far?", additional_kwargs={}, response_metadata={})],
 'response': "Hello, James! It's great to meet you! How's your day going so far?"}

Looks good so far, let's continue with a few more messages:

In [67]:
for msg in [
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    chain.invoke({"input": msg})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello, James! It's great to meet you! How's your day going so far?", additional_kwargs={}, response_metadata={})]
Human: I'm researching the different types of conversational memory.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content='The human, nam

We can see with each new message the initial `SystemMessage` is updated with a new summary of the conversation. This initial `SystemMessage` is then followed by the most recent `AIMessage` and `HumanMessage` objects.

### `ConversationSummaryBufferMemory` with `RunnableWithMessageHistory`

As with the previous memory types, we will implement this memory type again using the `RunnableWithMessageHistory` class. In our implementation we will modify the buffer window to be based on the number of messages rather than number of tokens. This tweak will make our implementation more closely aligned with original buffer window.

We will implement all of this via a new `ConversationSummaryBufferMessageHistory` class.

In [5]:
class ConversationSummaryBufferMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)
    k: int = Field(default_factory=int)

    def __init__(self, llm: ChatOpenAI, k: int):
        super().__init__(llm=llm, k=k)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages and summarizing the messages that we
        drop.
        """
        existing_summary: SystemMessage | None = None
        old_messages: list[BaseMessage] | None = None
        # see if we already have a summary message
        if len(self.messages) > 0 and isinstance(self.messages[0], SystemMessage):
            print(">> Found existing summary")
            existing_summary = self.messages.pop(0)
        # add the new messages to the history
        self.messages.extend(messages)
        # check if we have too many messages
        if len(self.messages) > self.k:
            print(
                f">> Found {len(self.messages)} messages, dropping "
                f"oldest {len(self.messages) - self.k} messages.")
            # pull out the oldest messages...
            old_messages = self.messages[:self.k]
            # ...and keep only the most recent messages
            self.messages = self.messages[-self.k:]
        if old_messages is None:
            print(">> No old messages to update summary with")
            # if we have no old_messages, we have nothing to update in summary
            return
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible. However, we need to keep "
                "the summary concise, the summary should be no longer than 1 paragraph."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{old_messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=existing_summary,
                old_messages=old_messages
            )
        )
        print(f">> New summary: {new_summary.content}")
        # prepend the new summary to the history
        self.messages = [SystemMessage(content=new_summary.content)] + self.messages

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

Redefine the `get_chat_history` function to use our new `ConversationSummaryBufferMessageHistory` class.

In [6]:
chat_map = {}
def get_chat_history(session_id: str, llm: ChatOpenAI, k: int) -> ConversationSummaryBufferMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryBufferMessageHistory(llm=llm, k=k)
    # return the chat history
    return chat_map[session_id]

Setup our pipeline with new configurable fields.

In [15]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

Finally, we invoke our runnable:

In [76]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123_with_summary_and_k_last_messages", "llm": llm, "k": 4}
)
chat_map["id_123_with_summary_and_k_last_messages"].messages

>> No old messages to update summary with


[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! How can I assist you today?', additional_kwargs={}, response_metadata={})]

In [77]:
for i, msg in enumerate([
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]):
    print(f"---\nMessage {i+1}\n---\n")
    pipeline_with_history.invoke(
        {"query": msg},
        config={"session_id": "id_123_with_summary_and_k_last_messages", "llm": llm, "k": 4}
    )

---
Message 1
---

>> No old messages to update summary with
---
Message 2
---

>> Found 6 messages, dropping oldest 2 messages.
>> New summary: James introduced himself and mentioned he is researching different types of conversational memory. The AI responded by outlining various types, including short-term, long-term, contextual, episodic, semantic, social memory, and conversational memory in AI, explaining their relevance in human communication and artificial intelligence. The AI also offered to help James explore any specific area of conversational memory further.
---
Message 3
---

>> Found existing summary
>> Found 6 messages, dropping oldest 2 messages.
>> New summary: James is researching different types of conversational memory, and the AI provided an overview of various types, including short-term, long-term, contextual, episodic, semantic, social memory, and conversational memory in AI. James specifically inquired about ConversationBufferMemory and ConversationBufferWindowMe

There we go, we've successfully implemented the `ConversationSummaryBufferMemory` type using `RunnableWithMessageHistory`!

In [16]:
# Random place for me to do invoking instead of using chatgpt website
# Do the following, but with user input for the feild query
# pipeline_with_history.invoke(
#     {"query": "Hi, my name is James"},
#     config={"session_id": "id_123_with_summary_and_k_last_messages", "llm": llm, "k": 4}
# )
# chat_map["id_123_with_summary_and_k_last_messages"].messages

while True:
    theinput = input("Enter your query (type 'exit', 'quit', or 'q' to quit):\n")
    if theinput.strip().lower() in ["exit", "quit", "q"]:
        print("Exiting chat.")
        break
    response = pipeline_with_history.invoke(
        {"query": theinput},
        config={"session_id": "id_123_with_summary_and_k_last_messages_in_a_loop", "llm": llm, "k": 4}
    )
    print("User:", theinput)
    print("AI:", response.get("First-Response", "No response"))

>> No old messages to update summary with
AI: Hello! How can I assist you today?


Error in RootListenersTracer.on_chain_end callback: NameError("name 'SystemMessage' is not defined")


AI: Nice to meet you, John! How can I help you today?


Error in RootListenersTracer.on_chain_end callback: NameError("name 'SystemMessage' is not defined")


AI: Of course! What do you need help with?


...

AI: I don't know your name yet! If you'd like to share it, feel free to do so. How can I help you today?


Error in RootListenersTracer.on_chain_end callback: NameError("name 'SystemMessage' is not defined")


AI: Certainly! Here’s a well-crafted response that showcases your understanding of Gemini Diffusion's capabilities and how you plan to leverage them:

---

I am particularly excited about the transformative potential of Gemini Diffusion in enhancing both real-time coding workflows and collaborative content generation. The model's impressive speed, reportedly around 1500 tokens per second, is a game-changer for interactive applications. For instance, I envision using Gemini Diffusion to accelerate my coding process by providing near-instantaneous refactoring suggestions and debugging assistance directly within my IDE. This capability would not only streamline my workflow but also foster a more dynamic coding environment where I can iterate on solutions rapidly.

Moreover, the iterative refinement process of Gemini Diffusion is something I find incredibly valuable. In collaborative writing projects, its ability to maintain global coherence while allowing for seamless editing and version 

Error in RootListenersTracer.on_chain_end callback: NameError("name 'SystemMessage' is not defined")


AI: You're welcome! If you have any questions in the future, feel free to ask. Goodbye!
Exiting chat.
